# Script de predicción 

### Índice   
1. [Carga de datos](#id1)

    1.1 [Valores nulos en los datos](#id11)
    
    1.2 [Las columnas READINGTHOUSANTH y DELTATHOUSANDTH](#id12)
    
    1.3 [La columna SAMPLETIME](#id13)
    
   
2. [Procesado de datos: las columnas DELTA y READING](#id2)

    2.1 [Agregar los consumos por fechas](#id21)
    
    2.2 [Completar los consumos faltantes](#id22)
    
    2.3 [Tipos de contadores según su serie temporal de consumos](#id23)
    
    2.4 [Tratar outliers en los consumos](#id24)
    
    

In [29]:
import pandas as pd
from tqdm import tqdm
import datetime
import math

## Carga de datos<a name="id1"></a>

Cargamos los datos y observamos que tenemos

In [4]:
df = pd.read_csv('../data/Modelar_UH2022.txt',sep = '|') 
df.head()

,ID,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH
0,0,2019-06-13 08:34:09,369320,0.0,17,0.0
1,0,2019-06-13 17:34:10,369403,0.0,2,0.0
2,0,2019-06-13 18:34:10,369403,0.0,0,0.0
3,0,2019-06-13 04:34:10,369284,0.0,1,0.0
4,0,2019-06-13 14:34:10,369356,0.0,28,0.0


Vemos el tamaño de los datos:

In [5]:
df.shape

(21404828, 6)

### Valores nulos en los datos<a name="id11"></a>

Vemos si hay valores nulos en los datos:

In [6]:
df.isna().sum()

ID                        0
SAMPLETIME                0
READINGINTEGER            0
READINGTHOUSANDTH    140056
DELTAINTEGER              0
DELTATHOUSANDTH      140056
dtype: int64

Como los valores nulos pertenecen a la parte de las milésimas los despreciamos y los imputamos a 0:

In [7]:
df = df.fillna(0)

### Las columnas READINGTHOUSANDTH y DELTATHOUSANDTH<a name="id12"></a>

Examinemos las columnas READINGTHOUSANDTH y DELTATHOUSANDTH:

In [15]:
print('Number READINGTHOUSANTH != 0:',(df['READINGTHOUSANDTH']!=0).sum())
print('Number DELTATHOUSANTH != 0:',(df['READINGTHOUSANDTH']!=0).sum())
print('Minimum READINGTHOUSANTH:',df['READINGTHOUSANDTH'].min())
print('Minimum DELTATHOUSANTH:',df['DELTATHOUSANDTH'].min())
print('Maximum READINGTHOUSANTH:',df['READINGTHOUSANDTH'].max())
print('Maximum DELTATHOUSANTH:',df['DELTATHOUSANDTH'].max())

Number READINGTHOUSANTH != 0: 2428718
Number DELTATHOUSANTH != 0: 2428718
Minimum READINGTHOUSANTH: 0.0
Minimum DELTATHOUSANTH: 0.0
Maximum READINGTHOUSANTH: 99.0
Maximum DELTATHOUSANTH: 99.0


Aunque el valor máximo para estas columnas es 99, el nombre 'Thousandth' deja claro que estas columnas hacen referencia a las milésimas por lo que dividiremos su valor entre 1000 y se lo agregaremos a la parte de las unidades. No obstante, no creemos que estas dos columnas vayan a tener un gran impacto en las predicciones ya que como máximo estaremos añadiendo 0,099 litros a la medición.

Tras agregarlo a las columnas de las unidades nos desharemos tanto de las columnas de las unidades como de las de las milésimas, quedándonos con el agregado:

In [21]:
df['DELTA'] = [j+(i/1000) for (j,i) in zip(df['DELTAINTEGER'].values, df['DELTATHOUSANDTH'].values)]
df['READING'] = [j+(i/1000) for (j,i) in zip(df['READINGINTEGER'].values, df['READINGTHOUSANDTH'].values)]
df.drop(['READINGINTEGER','READINGTHOUSANDTH','DELTAINTEGER','DELTATHOUSANDTH'], axis=1, inplace=True)
df.head()

,ID,SAMPLETIME,DELTA,READING
0,0,2019-06-13 08:34:09,17.0,369320.0
1,0,2019-06-13 17:34:10,2.0,369403.0
2,0,2019-06-13 18:34:10,0.0,369403.0
3,0,2019-06-13 04:34:10,1.0,369284.0
4,0,2019-06-13 14:34:10,28.0,369356.0


### La columna SAMPLETIME<a name="id13"></a>

La columna SAMPLETIME hace referencia al día y la hora a la que las medidas READING y DELTA fueron tomadas. Para poder trabajar comodamente con esta columna vamos a convertirla de tipo *string* a tipo *datetime* y a extraer de ella una nueva columna *DATE* en la que solo tendremos la fecha de la medida (sin la hora).

In [25]:
def get_fecha(date):
    return date.strftime("%Y-%m-%d")

def str2date(string):
    return datetime.datetime.strptime(string, '%Y-%m-%d %H:%M:%S')

In [26]:
tqdm.pandas()
df['SAMPLETIME'] = df['SAMPLETIME'].progress_apply(str2date)
df['DATE'] = df['SAMPLETIME'].progress_apply(get_fecha)
df.head()

100%|███████████████████████████████████████████████████████████████████| 21404828/21404828 [05:06<00:00, 69813.81it/s]


,ID,SAMPLETIME,DELTA,READING,DATE
0,0,2019-06-13 08:34:09,17.0,369320.0,2019-06-13
1,0,2019-06-13 17:34:10,2.0,369403.0,2019-06-13
2,0,2019-06-13 18:34:10,0.0,369403.0,2019-06-13
3,0,2019-06-13 04:34:10,1.0,369284.0,2019-06-13
4,0,2019-06-13 14:34:10,28.0,369356.0,2019-06-13


Antes de continuar ordenamos el dataframe por *ID* y *SAMPLETIME* para que sea más interpretable que si está desordenado:

In [27]:
df = df.sort_values(['ID','SAMPLETIME']).reset_index(drop=True)

## Procesado de datos: las columnas DELTA y READING<a name="id2"></a>

Antes de empezar con el tratamiento de las columnas DELTA y READING hay que resaltar que la mayoría de las decisiones que vamos a tomar en este apartado están justificadas por las conclusiones obtenidas en el script de exploración por lo que no nos detendremos a explicar como hemos llegado a ellas (para ello mejor ver el script de exploración).

En primer lugar vamos a definir una función que dadas dos fechas nos devuelva una lista de todas las fechas intermedias. Está función será una utilidad que emplearemos múltiples veces a lo largo de este notebook.

In [31]:
'''
given a start date in datetime format "start_date" and an "end_date" returns a list of strings with the dates from
"start_date" to "end_date".

Example:

start_date = datetime.date(2019, 9 , 30)
end_date = datetime.date(2019, 10, 7)
get_date_range(start_date, end_date)
'''
def get_date_range(start_date, end_date):
    number_of_days = (end_date-start_date).days
    return [(start_date + datetime.timedelta(days = day)).isoformat() for day in range(number_of_days+1)]

### Agregar los consumos por fechas<a name="id21"></a>

En este punto el primer paso claro a dar es agregar por fecha los consumos. No obstante nos encontramos con un serio problema: faltan muchas medidas. Para solventar esto vamos a hacer uso de las dos columnas *DELTA* y *READING*. En un principio si todos los datos están correctos consideramos que READING es más fiable. De no ser así, recurrimos a DELTA para tratar de completar los datos. Seguiremos el siguiente algoritmo:

- IF no hay ninguna medida para un día:
    * De momento lo dejamos como **NONE** y lo completaremos más adelante
- IF hay 24 medidas, es decir, está completo:
    * IF hay 24 medidas para el día anterior:
        + Tomamos $max(READING_{actual})-max(READING_{anterior})$ 
    * IF no hay 24 medidas para el día anterior, es decir, está incompleto:
        + Tomamos $sum(DELTA_{actual})$
- IF no hay 24 medidas, es decir, está incompleto
    * Tomamos $24/N_{medidas}*sum(DELTA_{actual})$, es decir, calculamos el promedio de las medidas que haya y lo múltiplicamos por 24 (como si hubiese 24 medidas).

In [32]:
start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 31)
complete_year = get_date_range(start_date, end_date)

delta_df = pd.DataFrame(columns=df['ID'].unique(), index =complete_year)

#primero rellenamos la primera columna
date = complete_year[0]
for i in tqdm(df['ID'].unique()):
    one_counter = df[df['ID']==i]
    # si no hay ninguna medida
    if len(one_counter[one_counter['DATE']==date]) == 0:
        delta_df[i][date] = None
    # si el contador está completo para ese dia
    elif len(one_counter[one_counter['DATE']==date]) >= 24:
        delta_df[i][date] = one_counter[one_counter['DATE']==date]['DELTA'].sum()
    # si el contador no está completo para ese dia
    else:
        delta_df[i][date] = (24/len(one_counter[one_counter['DATE']==date])) * \
                         one_counter[one_counter['DATE']==date]['DELTA'].sum()

100%|██████████████████████████████████████████████████████████████████████████████| 2747/2747 [02:17<00:00, 19.91it/s]


In [33]:
for i in tqdm(df['ID'].unique()):
    one_counter = df[df['ID']==i]
    for j in range(1, len(complete_year)):
        date = complete_year[j]
        # si no hay ninguna medida
        if len(one_counter[one_counter['DATE']==date]) == 0:
            delta_df[i][date] = None
        # si el contador está completo para ese dia
        elif len(one_counter[one_counter['DATE']==date]) >= 24:
            # si el contador está completo para el dia anterior
            if len(one_counter[one_counter['DATE']==complete_year[j-1]]) >= 24:
                delta_df[i][date] = one_counter[one_counter['DATE']==complete_year[j]]['READING'].max() - \
                                    one_counter[one_counter['DATE']==complete_year[j-1]]['READING'].max()
            # si el contador no está completo para el dia anterior
            else:
                delta_df[i][date] = one_counter[one_counter['DATE']==date]['DELTA'].sum()
        # si el contador no está completo para ese dia
        else:
            delta_df[i][date] = (24/len(one_counter[one_counter['DATE']==date])) * \
                             one_counter[one_counter['DATE']==date]['DELTA'].sum()

100%|████████████████████████████████████████████████████████████████████████████| 2747/2747 [2:07:09<00:00,  2.78s/it]


Tras aplicar este algoritmo nos quedamos con un dataframe en el que hemos completado todos los días para los que había al menos 1 medida. Sin embargo, los días para los que no había ninguna medida siguen siendo **NONE**. Por lo que necesitaremos otras estrategias para completarlos.

### Completar medidas faltantes<a name="id22"></a>

Empezaremos por completar los **NONE** comprendidos entre dos días para los cuales sí se tienen medidas, es decir, que no están ni al principio ni al final de la serie temporal. Para ello tomaremos la última medida *READING* del día en el que empieza la secuencia de **NONE** y la primera medida *READING* del día que termina la secuencia. Si restamos estas dos medidas obtendremos el consumo de agua total durante los días sin medidas, es decir, durante la secuencia de **NONE**. Para interpolar entre estas dos medidas dividiremos este consumo total entre el número de días sin medidas.

In [34]:
delta_df.to_pickle('../data/counters_in_rows_2.pkl')

In [116]:
delta_df2 = pd.read_pickle('../data/counters_in_rows_2.pkl')

In [112]:
def complete_huecos(deltas, readings):
    consecutive_nones = 0
    last_date_not_none = None
    none_dates = []
    for date in deltas.index:
        if deltas[date] == None:
            consecutive_nones += 1
            none_dates.append(date)
        elif deltas[date] != None:
            if consecutive_nones > 0:
                begin = readings[readings['DATE']==last_date_not_none]['READING'].max()
                end = readings[readings['DATE']==date]['READING'].min()
                for date_2 in none_dates:
                    deltas[date_2] = (end-begin)/consecutive_nones
            consecutive_nones = 0
            last_date_not_none = date  
            none_dates = []
    return deltas

In [113]:
for i in tqdm(df['ID'].unique()): 
    delta_df[i] = complete_huecos(delta_df[i], df[df['ID']==i][['DATE', 'READING']])

100%|██████████████████████████████████████████████████████████████████████████████| 2747/2747 [17:44<00:00,  2.58it/s]


### Tipos de contadores según su serie temporal de consumos<a name="id23"></a>

Tras este procesado tenemos tres tipos de contadores:
- Contadores completos
- Contadores a los que les faltan medidas al principio
- Contadores a los que les faltan medidas al final

Vamos a dividir de manera más formal los contadores en tipos según su serie temporal de consumos para los cuales seguiremos distintas estrategias de predicción:
- **Tipo 1:** Contadores cuyas medidas sean todas 0.
- **Tipo 2:** Contadores completos al menos en Enero de 2020 no pertenecientes al Tipo 1.
- **Tipo 3:** Contadores sin medidas en Noviembre, Diciembre de 2019 y Enero de 2020 pero con medidas en Febrero de 2019 no pertenecientes al Tipo 1.
- **Tipo 0:** Contadores no pertencientes a ninguno de los anteriores tipos.